# Penguin Analysis with jupyter-cc

This notebook demonstrates using `%cc` magic to work with Claude Code inside Jupyter.
Data is loaded with regular Python — `%cc` is used for analysis and visualization.

In [ ]:
%load_ext jupyter_cc_magic

## Load Data

In [ ]:
import seaborn as sns

df = sns.load_dataset("penguins").dropna()
df.head()

## Visualize

Ask Claude to create a scatter plot from the loaded data.

In [ ]:
%cc Create a scatter plot of bill_length_mm vs bill_depth_mm, colored by species. Use matplotlib.

In [ ]:
# [CC] Scatter plot: bill_length_mm vs bill_depth_mm colored by species

import matplotlib.pyplot as plt
from IPython.utils.capture import capture_output

colors = {"Adelie": "#E76F51", "Chinstrap": "#2A9D8F", "Gentoo": "#457B9D"}

with capture_output() as _claude_captured_output:
    fig, ax = plt.subplots(figsize=(8, 5))
    for species, group in df.groupby("species"):
        ax.scatter(
            group["bill_length_mm"], group["bill_depth_mm"], label=species, color=colors[species], alpha=0.7, s=50
        )
    ax.set_xlabel("Bill Length (mm)")
    ax.set_ylabel("Bill Depth (mm)")
    ax.set_title("Bill Length vs Bill Depth by Species")
    ax.legend(title="Species")
    plt.tight_layout()
    plt.show()

for output in _claude_captured_output.outputs:
    display(output)

## Summary Statistics

Start a fresh conversation to calculate summary stats.

In [ ]:
%cc_new Calculate summary statistics for body_mass_g grouped by species and sex

## In-place Editing

Use `%cc_cur` to replace the prompt cell with generated code.

In [ ]:
%cc_cur Create a function that takes a species name and returns a dict of mean measurements from df